# Palmer Amaranth (Amaranthus palmeri) Growth Stage Detection
This Google Colab notebook has been adapted from the **official YOLOv3 notebook** by **Ultralytics** to train and evaluate YOLOv3 on the Palmer amaranth Growth Stage (PAGS8) dataset. The dataset is available for download from [Weed-AI](). Prior to use, the dataset should be setup using the instructions from the official repository.

It accompanies the preprint: **Multi-growth stage plant recognition: a case study of Palmer amaranth (Amaranthus palmeri) in cotton (Gossypium hirsutum)** available on [arXiv](https://arxiv.org/abs/2307.15816). Please consider citing this preprint if you use the work in your research.

Models were trained using Google Colab Pro+ with access to an NVIDIA A100-SXM4-40GB.

This notbook consists of five parts:
1. Setup
2. Training
3. Evaluation
4. Inference
5. Additional information

For additional documentation, training options and inference procedures please visit the official Ultralytics repository. (Remember to choose GPU in Runtime if not already selected. Runtime --> Change Runtime Type --> Hardware accelerator --> GPU)

<img src="https://github.com/geezacoleman/Palmer-detection/assets/51358498/6040227e-a072-43bf-a789-72e0833f3168"/>

# Setup

Clone repo, install dependencies and check PyTorch and GPU.

In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Assumes you have already cloned the Palmer-detection repository
%cd '/content/drive/MyDrive/Palmer-detection'

from datetime import datetime

DATE = datetime.now().strftime('%Y%m%d')
IMAGE_SIZE = 1280

In [ ]:
%cd yolov3
%pip install -qr requirements.txt  # install requirements


# 2. Training
Train a YOLOv3 model on the [COCO128](https://www.kaggle.com/ultralytics/coco128) dataset with `--data coco128.yaml`, starting from pretrained `--weights yolov3.pt`, or from randomly initialized `--weights '' --cfg yolov3yaml`.

- **Pretrained [Models](https://github.com/ultralytics/yolov3/tree/master/models)** are downloaded
automatically from the [latest YOLOv3 release](https://github.com/ultralytics/yolov3/releases)
- **[Datasets](https://github.com/ultralytics/yolov3/tree/master/data)** available for autodownload include: [COCO](https://github.com/ultralytics/yolov3/blob/master/data/coco.yaml), [COCO128](https://github.com/ultralytics/yolov3/blob/master/data/coco128.yaml), [VOC](https://github.com/ultralytics/yolov3/blob/master/data/VOC.yaml), [Argoverse](https://github.com/ultralytics/yolov3/blob/master/data/Argoverse.yaml), [VisDrone](https://github.com/ultralytics/yolov3/blob/master/data/VisDrone.yaml), [GlobalWheat](https://github.com/ultralytics/yolov3/blob/master/data/GlobalWheat2020.yaml), [xView](https://github.com/ultralytics/yolov3/blob/master/data/xView.yaml), [Objects365](https://github.com/ultralytics/yolov3/blob/master/data/Objects365.yaml), [SKU-110K](https://github.com/ultralytics/yolov3/blob/master/data/SKU-110K.yaml).
- **Training Results** are saved to `runs/train/` with incrementing run directories, based on the parameters you set.


### Weights and Biases
WandB is an excellent tool to track/monitor training performance, particularly across large training runs. While optional, it is recommended, however, requires setting up an account and logging in below.

Only run this cell if you plan on using WandB.

In [ ]:
# Weights & Biases  (optional)
%pip install -q wandb
import wandb
wandb.login()

In [ ]:
## Training all models, folds and classes
models = ['-tiny', '', '-spp'] # all three classes
classes = ['1', '8']
folds = [0, 1, 2, 3, 4]

# warning - running this will train 3 x 2 x 5 models (30 total)
for model in models:
  for class_num in classes:
    for fold in folds:
      yaml_path = f'{class_num}cls_fold_{fold}.yaml'
      full_name = f'{DATE}_yolov3{model}_B8_F0_{model}_{class_num}'
      
      !python train.py --img {IMAGE_SIZE} --cfg yolov3{model}.yaml --hyp hyp.scratch-low.yaml --batch 8 --epochs 30 --data data/{yaml_path} --weights yolov3{model}.pt --name {full_name}


# 3. Evaluation
This will run over all the runs within the train directory and run the evaluation on them.

In [ ]:
import os
from tqdm import tqdm

%cd '/content/drive/My Drive/Palmer-detection/yolov3'
train_path = "/content/drive/My Drive/Palmer-detection/yolov3/runs/train"

for run_name in tqdm(os.listdir(train_path)):
  if 'v3' not in run_name:
    print(f'skipping {run_name}')
  
  class_num = run_name.split("_")[4]
  fold = run_name.split("_")[3][-1]
  yaml_path = f"{class_num}cls_fold_{fold}.yaml"
    
  !python val.py --img {IMAGE_SIZE} --weights runs/train/{run_name}/weights/best.pt --data data/{yaml_path} --name {run_name} --task 'test' --batch-size 8 --iou-thres 0.6 --conf-thres 0.001 --verbose --save-metrics


# 4. Inference

In [ ]:
%cd '/content/drive/My Drive/Palmer-detection/yolov5'
models = ['-tiny', '', '-spp']
yaml_path = f"fold_0_pa_8cls.yaml"
source = r"datasets/test"

for model in models:
  run_name = f'{DATE}_yolov5{model}5_B8_F0_8cls'
  !python detect.py --img {IMAGE_SIZE} --weights runs/train/done/{run_name}/weights/best.pt --source {source} --name {run_name} --iou-thres 0.45 --conf-thres 0.25

# 5. Additional Information

## Local Logging

All results are logged by default to `runs/train`, with a new experiment directory created for each new training as `runs/train/exp2`, `runs/train/exp3`, if a name isn't provided. View train and val jpgs to see mosaics, labels, predictions and augmentation effects. Note an Ultralytics **Mosaic Dataloader** is used for training (shown below), which combines 4 images into 1 mosaic during training.

> <img src="https://user-images.githubusercontent.com/26833433/131255960-b536647f-7c61-4f60-bbc5-cb2544d71b2a.jpg" width="700">  
`train_batch0.jpg` shows train batch 0 mosaics and labels

> <img src="https://user-images.githubusercontent.com/26833433/131256748-603cafc7-55d1-4e58-ab26-83657761aed9.jpg" width="700">  
`test_batch0_labels.jpg` shows val batch 0 labels

> <img src="https://user-images.githubusercontent.com/26833433/131256752-3f25d7a5-7b0f-4bb3-ab78-46343c3800fe.jpg" width="700">  
`test_batch0_pred.jpg` shows val batch 0 _predictions_

Training results are automatically logged to [Tensorboard](https://www.tensorflow.org/tensorboard) and [CSV](https://github.com/ultralytics/yolov5/pull/4148) as `results.csv`, which is plotted as `results.png` (below) after training completes. You can also plot any `results.csv` file manually:

```python
from utils.plots import plot_results 
plot_results('path/to/results.csv')  # plot 'results.csv' as 'results.png'
```
